In [1]:
import pandas as pd
from sqlalchemy import create_engine

def connect_to_db(user, password, host, port, database):
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}'
    engine = create_engine(connection_string)
    return engine

In [ ]:
# 2
def rentals_month(engine, month, year):
    query = f"""
    SELECT * FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    df = pd.read_sql(query, engine)
    return df

In [ ]:
# 3
def rental_count_month(df, month, year):
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_count

In [ ]:
# 4
def compare_rentals(df1, df2):
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    combined_df['difference'] = combined_df.iloc[:, 1] - combined_df.iloc[:, 2]
    return combined_df

In [ ]:
# Establish a connection to the database
user = 'root'
password = 'password'
host = 'localhost'
port = '3306'
database = 'sakila'

engine = connect_to_db(user, password, host, port, database)

# Retrieve rental data for May and June
may_rentals = rentals_month(engine, 5, 2005)
june_rentals = rentals_month(engine, 6, 2005)

# Calculate the rental counts for May and June
may_rental_counts = rental_count_month(may_rentals, 5, 2005)
june_rental_counts = rental_count_month(june_rentals, 6, 2005)

# Compare the rental counts
comparison_df = compare_rentals(may_rental_counts, june_rental_counts)

# Display the comparison DataFrame
comparison_df.head()

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,-5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,-2.0
3,4,0.0,6.0,-6.0
4,5,3.0,5.0,-2.0
